# Introducing Aurite Agents

Welcome to the Aurite Agents framework! This tutorial is the first in a series designed to guide you through building and running your own AI agents. 

**Goal:** By the end of this notebook, you will have configured and run a simple AI agent that can answer a question using a powerful Large Language Model (LLM).

**What is Aurite Agents?** It's a Python framework that provides the building blocks to create, manage, and deploy AI agents. It handles the complex parts of interacting with LLMs and external tools, so you can focus on defining what your agent does.

**Prerequisites:**

*   Python 3.8+ installed (Google Colab uses Python 3.11)

*   An API key from an LLM provider (we'll use OpenAI in this example).

## Install the aurite Python package

In [1]:
%pip install aurite==0.3.3

Note: you may need to restart the kernel to use updated packages.


## Configure Your API Key

To use a Large Language Model like OpenAI's GPT-4, you need an API key. For security, it's best practice to set this as an environment variable rather than writing it directly in your code.

**Google Colab Users:** Ensure your Google Colab **Secrets** tab (the key icon on the left sidebar) has an `OPENAI_API_KEY` set and enable it for this notebook.

**IDE Users:** Ensure you have a .env file containing a valid `OPENAI_API_KEY` in your root vscode folder

In [2]:
import os

# if this notebook is running in Google Colab, we can use the userdata module to fetch secrets
# Note: This requires the user to have set up their secrets in Colab's secret manager.

try:
    from google.colab import userdata #type: ignore
    # Fetch the secret value from Colab's secret manager and set it as an environment variable so your script can find it
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
except ImportError: # This is not a real error, just a check if running in Colab
    from dotenv import load_dotenv
    load_dotenv()  # Load environment variables from a .env file if it exists
except Exception as e:
    print(f"Error fetching OPENAI_API_KEY from Colab's secret manager: {e}")

print("✅ OpenAI API Key is set!")

✅ OpenAI API Key is set!


## Initialize Aurite

This is the main class you will be working with. You can use the `Aurite` class to register and execute LLMs, MCP Servers, Agents, and Simple/Custom Workflows.

The `initialize()` method will become more relevant later when you learn about `Project Files`, but for now this method will simply load the built-in configurations provided by the package.

In [3]:
from aurite import Aurite

# This is the main entry point for your Aurite project.
# You can use this class to register and execute LLMs, MCP Servers, Agents, Simple Workflows, and Custom Workflows.
aurite = Aurite()

# This next command will start up the MCP Tool servers, along with loading your project json file.
await aurite.initialize()

WARNING  [aurite.config.component_manager] Duplicate component ID 'planning_server' (from file /home/wilcoxr/workspace/aurite/aurite-agents/notebook-venv/lib/python3.11/site-packages/aurite/packaged/component_configs/mcp_servers/example_mcp_servers.json) for type 'mcp_servers'. Packaged default will be kept. User project component will override if loaded later with override flag.
INFO     [aurite.config.component_manager] User project config directory not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/config. No project-specific components will be loaded.
WARNING  [aurite.config.project_manager] Project configuration file not found: /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json
WARNING  [aurite.config.project_manager] Project file not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json. Creating an empty project named 'aurite_config.json'.
INFO     [aurite.host.host] MCP Host initialization attem

## Building Your First Agent 

Let's build an **`Agent`**

In order to create an agent, the only required variable is a `name` (so we can identify the agent for registration and execution).

Since our agent has no tools, and no system prompt, we are essentially creating `ChatGPT`.

In [4]:
from aurite import AgentConfig

# Create an agent configuration with the name "My First Agent"
agent_config = AgentConfig(name="My First Agent")

# Let's register the agent with Aurite.
await aurite.register_agent(agent_config)

### Run the Agent!

With everything set up, we can now ask our agent a question. We'll provide our Agent (which is just ChatGPT) with a `User Message` just like how you send a message to ChatGPT on their website.

In [5]:
# Write a user message to send to the agent.
user_message = "Hello ChatGPT! Can you tell me a joke?"

# Run the agent with the user's query
agent_result = await aurite.run_agent(
    agent_name="My First Agent",
    user_message=user_message
)

# Print the agent's response
print(agent_result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'My First Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-4-turbo-preview using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'My First Agent'...
INFO     [httpx] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     [aurite.execution.facade] Facade: Aurite Agent 'My First Agent' conversation finished.


Sure! Here's one for you:

Why can't you give Elsa a balloon?

Because she will let it go!


## Assign your Agent with a Role and a Task

The previous example is the "code version" of a ChatGPT conversation. Now it is time to build a real agent.

The first and most important part of an Agent is its' **`System Prompt`**

### What is a System Prompt?
**A `System Prompt` is a textual description explaining how the LLM (ChatGPT) should respond to user messages.**

Think of your agent as if they are an improv actor in a play. As the director of this play, you need to tell your actor what their role is. The level of detail in your explanation depends on how specific the role is.

### Crafting a System Prompt
Here is a useful template for creating system prompts:

- Role: "You are a..."
- Task: "Your job is to.."
- Context: "In order to do this..."
- Rules: "You MUST use..."

In [6]:
# Let's create a new agent configuration for a dramatic agent with a system prompt.

# This agent will respond in an overly dramatic tone. The \ characters just allow us to break the text into multiple lines for readability.
system_prompt = "You are a helpful but overly dramatic assistant. \
Your job is to answer the user's question in an overly dramatic tone. \
You must only respond with the answer to the user's question."

# Create a new agent configuration and set the system prompt
dramatic_agent = AgentConfig(
    name="My Dramatic Agent",
    system_prompt=system_prompt
)

# Register your new Dramatic Agent with Aurite
await aurite.register_agent(dramatic_agent)

# Run the new Dramatic Agent with the same user message
agent_result = await aurite.run_agent(
    agent_name="My Dramatic Agent",
    user_message=user_message # Same as before. Notice the agent's response will be different due to the new system prompt.
)

# Print the dramatic agent's response
print(agent_result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'My Dramatic Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-4-turbo-preview using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'My Dramatic Agent'...
INFO     [httpx] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     [aurite.execution.facade] Facade: Aurite Agent 'My Dramatic Agent' conversation finished.


Oh, brace yourself, for I am about to unveil a jest so whimsical, so utterly delightful, that it may very well send ripples of laughter through the very core of your being! Ready yourself, for here it comes: 

Why don't skeletons fight each other? They don't have the guts! 

Ah, the sweet symphony of humor! Can you feel the earth tremble with mirth? Are the stars not twinkling just a bit brighter with glee? Ah, what a moment we've shared!


## Give your Agent access to Tools

### What are Tools?
**Tools are functions (executable code) that provide a description of the code to be executed.**

The Agent (which is just ChatGPT as we have discussed) will use this description to understand how to call the tool (how to execute the code).

This is what truly defines an Agent. The literal definition of Agency is **the ability to act**. Without tools, ChatGPT can only ever talk to you. Tools give ChatGPT the ability to perform tasks by executing code in a live environment.

### How Tools are Defined
The Aurite Agents framework uses MCP (Model Context Protocol) to define Tools.

You can write these MCP Tool Servers yourself, but the framework has provided many built-in tool servers for you, some we created and others we found on MCP Server Directories.

All you need to do is provide the name(s) of the tool servers that you want to give your agent access to (as an `"mcp_servers"` list seen below)

We will also craft a system prompt explaining how the agent should use the tools. This is why system prompts are so important.

In [7]:
# This agent will respond in an overly dramatic tone.
system_prompt = "You are a Weather Forecaster. \
Your job is to provide accurate weather information based on the user's query. \
In order to do this, you must use the weather tool at your disposal to gather the necessary information. \
You must only respond with the weather forecast for the location specified by the user."

tools = ["weather_server"]  # This is a "fake" built-in tool for weather information.

#! This specific tool server returns a pre-defined weather forecast from New York for testing purposes. Do not change the city in the user message.
user_message = "What's the weather like in New York?"

# 3. Define and register an Agent configuration
agent_config = AgentConfig(
    name="My Weather Agent",
    system_prompt=system_prompt,
    mcp_servers=tools
)

# Register the agent with Aurite
await aurite.register_agent(agent_config)

# Run the weather agent with a user query
agent_result = await aurite.run_agent(
    agent_name="My Weather Agent",
    user_message="What's the weather like in New York?"
)

# Print the weather agent's response
print(agent_result.primary_text)

INFO     [aurite.host.host] Attempting to dynamically register client: weather_server
INFO     [aurite.host.host] Client 'weather_server' dynamically registered and initialized successfully.
WARNING  [aurite.execution.facade] Facade: Agent 'My Weather Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-4-turbo-preview using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'My Weather Agent'...
INFO     [httpx] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     [aurite.host.host] Executing tool 'weather_lookup' on client 'weather_server' for agent 'My Weather Agent'
INFO     [httpx] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     [aurite.execution.facade] Facade: Aurite Agent 'My Weather Agent' conversat

The weather in New York is currently 22°C with partly cloudy skies and a humidity of 60%.


## Congratulations!

You have successfully built and run your first AI agent using the Aurite framework. 

**Recap:**

- You learned how to configure an `Agent` with a `System Prompt` (system_prompt) and a set of `Tools` (mcp_servers).

- You initialized the `Aurite` application and registered your components.

- You ran the agent with a query and received a response.

In the next tutorial, we will explore some real tool servers as we build agents to achieve different goals!